In [1]:
import pandas as pd
import numpy as np

def Allocated_Score(K, W, S):

    #Normalize score matrix
    ballots = pd.DataFrame(S.values/K, columns=S.columns)
    
    #Find number of voters and quota size
    V = ballots.shape[0]
    quota = V/W
    ballot_weight = pd.Series(np.ones(V),name='weights')
    
    #Populate winners in a loop
    winner_list = []
    while len(winner_list) < W:

        weighted_scores = ballots.multiply(ballot_weight, axis="index")

        #Select winner
        w = weighted_scores.sum().idxmax()
    
        #Add winner to list
        winner_list.append(w)

        #remove winner from ballot
        ballots.drop(w, axis=1, inplace=True)
    
        #Create lists for manipulation
        cand_df = pd.concat([ballot_weight,weighted_scores[w]], axis=1).copy() 
        cand_df_sort = cand_df.sort_values(by=[w], ascending=False).copy()  
        
        #find the score where a quota is filled
        split_point = cand_df_sort[cand_df_sort['weights'].cumsum() < quota][w].min()
    
        #Amount of ballot for voters who voted more than the split point
        spent_above = cand_df[cand_df[w] > split_point]['weights'].sum()
        
        #Allocate all ballots above split point
        if spent_above>0:    
            cand_df.loc[cand_df[w] > split_point, 'weights'] = 0.0
    
        #Amount of ballot for voters who gave a score on the split point
        weight_on_split = cand_df[cand_df[w] == split_point]['weights'].sum()

        #Fraction of ballot on split needed to be spent
        if weight_on_split>0:     
            spent_value = (quota - spent_above)/weight_on_split
    
            #Take the spent value from the voters on the threshold evenly
            cand_df.loc[cand_df[w] == split_point, 'weights'] = cand_df.loc[cand_df[w] == split_point, 'weights'] * (1 - spent_value)
    
        ballot_weight = cand_df['weights'].clip(0.0,1.0)

    return winner_list

In [ ]:
import itertools
# Compose Ballots
candidates = ['Squash', 'Potato',  'Broccoli', 'Blueberry', 'Strawberry', 'Banana']
#candidates = ['🥕', '🥔',  '🥦', '🫐', '🍓', '🍌']

ballot1 = {candidates[0]: 5, candidates[1]: 5, candidates[2]: 5, candidates[3]: 0, candidates[4]: 0, candidates[5]: 0}
ballot2 = {candidates[0]: 3, candidates[1]: 5, candidates[2]: 5, candidates[3]: 2, candidates[4]: 2, candidates[5]: 2}
ballot3 = {candidates[0]: 5, candidates[1]: 5, candidates[2]: 5, candidates[3]: 5, candidates[4]: 0, candidates[5]: 0}
ballot4 = {candidates[0]: 0, candidates[1]: 0, candidates[2]: 0, candidates[3]: 5, candidates[4]: 5, candidates[5]: 4}
ballot5 = {candidates[0]: 0, candidates[1]: 0, candidates[2]: 4, candidates[3]: 5, candidates[4]: 5, candidates[5]: 5}
ballot6 = {candidates[0]: 0, candidates[1]: 0, candidates[2]: 0, candidates[3]: 5, candidates[4]: 5, candidates[5]: 3}

# ballot_proportion = {ballot1: 112, ballot2: 6,
#  ballot3: 4, ballot4: 73, ballot5: 4, ballot6: 1}

ballots = list(itertools.repeat(ballot1, 112)) + \
    list(itertools.repeat(ballot2, 6)) + \
    list(itertools.repeat(ballot3, 4)) + \
    list(itertools.repeat(ballot4, 73)) + \
    list(itertools.repeat(ballot5, 4)) + \
    list(itertools.repeat(ballot6, 1))
ballots = pd.DataFrame(ballots)

Allocated_Score(K, W, S)